In [1]:
!gcloud auth login
!mkdir /content/images/
!gsutil -mq cp gs://lily-li-dataset/satire/**.jsonl /content/
!gsutil -mq cp -r gs://lily-li-dataset/satire/defaults/images/ /content/images/

In [2]:
import numpy as np
import jsonlines
from PIL import Image
import tensorflow as tf

with jsonlines.open('/content/train.jsonl', 'r') as data:
    hl_train = []
    img_train = []
    y_train = []
    for i, d in enumerate(data):
        try:
            filepath = "/content/images/images/" + str(i) + ".jpg"  
            img = Image.open(filepath).resize((128,128), Image.BICUBIC).convert('RGB')
            img_train.append(np.asarray(img) / 255.)
            hl_train.append(d["text"])
            y_train.append(d["label"])
        except:
            print("Image at filepath {0} does not exist".format(filepath))

with jsonlines.open('/content/test.jsonl', 'r') as data:
    hl_test = []
    img_test= []
    y_test = []
    for i, d in enumerate(data):
        try:
            filepath = "/content/images/images/" + str(i) + ".jpg"  
            img = Image.open(filepath).resize((128,128), Image.BICUBIC).convert('RGB')
            img_test.append(np.asarray(img) / 255.)
            hl_test.append(d["text"])
            y_test.append(d["label"])
        except:
            print("Image at filepath {0} does not exist".format(filepath))

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [3]:
from transformers import AutoTokenizer, TFBertModel 
max_len = 128
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_dict = tokenizer(list(hl_train), padding="max_length", truncation=True, max_length=max_len, return_tensors="tf")
train_input_ids, train_segment_ids, train_attention_masks = (train_dict["input_ids"],
                                                             train_dict["token_type_ids"],
                                                             train_dict["attention_mask"])
test_dict = tokenizer(list(hl_test), padding="max_length", truncation=True, max_length=max_len, return_tensors="tf")
test_input_ids, test_segment_ids, test_attention_masks = (test_dict["input_ids"],
                                                          test_dict["token_type_ids"],
                                                          test_dict["attention_mask"])

In [4]:
def create_model():
    encoder = TFBertModel.from_pretrained("bert-base-uncased")  

    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
    segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
    input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)

    bert_output = encoder(input_ids, token_type_ids=segment_ids, attention_mask=input_mask)[0]
    bert_output = tf.keras.layers.Flatten()(bert_output)
    ####################################
    ResNet = tf.keras.applications.ResNet101(
            include_top=False,
            weights="imagenet",
            input_shape=(128,128,3),
            )

    input_img = tf.keras.layers.Input(shape=(128,128,3))
    img_output = ResNet(input_img)
    img_output = tf.keras.layers.Flatten()(img_output)
    #########################################
    merged = tf.keras.layers.concatenate([bert_output, img_output], axis=-1)

    dense = tf.keras.layers.Dense(256, activation='relu')(merged)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    model = tf.keras.Model(
        inputs=[input_ids, segment_ids, input_mask, input_img],
        outputs=output,
    )
    return model

In [5]:
model = create_model()
model.summary()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 128, 128, 3) 0                                            
_______________________________________________________________________________________

In [6]:
img_train = np.stack(img_train, axis=0)
img_test = np.stack(img_test, axis=0)

img_train = tf.convert_to_tensor(img_train)
img_test = tf.convert_to_tensor(img_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [7]:
optimizer = tf.keras.optimizers.Adam(lr=8e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.binary_crossentropy,
              metrics=["accuracy", "AUC", "Precision", "Recall"])

In [8]:
model.fit(x=[train_input_ids, train_segment_ids, train_attention_masks, img_train],
           y=y_train, batch_size=16, epochs=5, validation_steps=2000//32,
           steps_per_epoch=8000//32,
           validation_data=([test_input_ids, test_segment_ids, test_attention_masks, img_test], y_test))

Epoch 1/5
250/250 [==============================] - 156s 626ms/step - loss: 0.4423 - accuracy: 0.8105 - auc: 0.8817 - precision: 0.7760 - recall: 0.7472 - val_loss: 0.2481 - val_accuracy: 0.8982 - val_auc: 0.9633 - val_precision: 0.8493 - val_recall: 0.9140
Epoch 2/5
250/250 [==============================] - 152s 610ms/step - loss: 0.2642 - accuracy: 0.8935 - auc: 0.9544 - precision: 0.8756 - recall: 0.8525 - val_loss: 0.2507 - val_accuracy: 0.9274 - val_auc: 0.9705 - val_precision: 0.9564 - val_recall: 0.8624
Epoch 3/5
250/250 [==============================] - 152s 609ms/step - loss: 0.1114 - accuracy: 0.9580 - auc: 0.9904 - precision: 0.9504 - recall: 0.9431 - val_loss: 0.2932 - val_accuracy: 0.9173 - val_auc: 0.9623 - val_precision: 0.9012 - val_recall: 0.8968
Epoch 4/5
250/250 [==============================] - 152s 610ms/step - loss: 0.1263 - accuracy: 0.9563 - auc: 0.9887 - precision: 0.9452 - recall: 0.9469 - val_loss: 0.2416 - val_accuracy: 0.9235 - val_auc: 0.9731 - val_pre